In [1]:
from thermoextrap import *
import xarray as xr
# ideal gas extrapolation
import thermoextrap.xtrapy.xpan_vol_ig as xpan_vol_ig
# "general extrapolation"
import thermoextrap.xtrapy.xpan_vol as xpan_vol

In [2]:
# test Data
# samples
n = int(1e5)
# number of x values
nv = 5
_u_offset = 0.0
_x_offset = 0.0
np.random.seed(0)
u = np.random.rand(n) + _u_offset
x = np.random.rand(n, nv) + _x_offset

In [3]:
#With original extrapolation package
class VolumeExtrapModelIG(ExtrapModel):
    """Class to hold information about an extrapolation in size for a 1D system (e.g. our ideal gas model)
    """
    #Can't go to higher order in practice, so don't return any symbolic derivatives
    #Instead, just use this to check and make sure not asking for order above 1
    def calcDerivFuncs(self):
        if self.maxOrder > 1:
            print('Volume extrapolation cannot go above 1st order without derivatives of forces.')
            print('Setting order to 1st order.')
            self.maxOrder = 1
        return None
    #And given data, calculate numerical values of derivatives up to maximum order
    #Will be very helpful when generalize to different extrapolation techniques
    #(and interpolation)
    def calcDerivVals(self, refL, x, W):
        """Calculates specific derivative values at B with data x and U up to max order.
        Returns these derivatives. Only go to first order for volume extrapolation. And
        here W represents the virial instead of the potential energy.
        """
        if x.shape[0] != W.shape[0]:
            print('First observable dimension (%i) and size of potential energy array (%i) don\'t match!'%(x.shape[0], W.shape[0]))
            return
        wT = np.array([W]).T
        avgX = np.average(x, axis=0)
        avgW = np.average(W)
        avgXW = np.average(x*wT, axis=0)
        derivVals = np.zeros((2, x.shape[1]))
        derivVals[0] = avgX
        derivVals[1] = (avgXW - avgX*avgW) / refL
        #Add the unique correction for the observable <x> in the ideal gas system
        #It turns out this is just <x> itself divided by L
        derivVals[1] += avgX / refL
        return derivVals

In [4]:
em = VolumeExtrapModelIG(maxOrder=1, refB=2.0, xData=x, uData=u)
xem = xpan_vol_ig.factory_extrapmodel(order=1, volume=2.0, uv=u, xv=x)

In [5]:
volumes = [0.1, 0.5, 1.5, 2.0]

In [6]:
np.testing.assert_allclose(em.predict(volumes), xem.predict(volumes))

In [7]:
from importlib import reload
reload(xpan_vol)

<module 'thermoextrap.xtrapy.xpan_vol' from '/Users/wpk/Documents/python/projects/thermodynamic-extrapolation/thermoextrap/xtrapy/xpan_vol.py'>

In [8]:
xem2 = xpan_vol.factory_extrapmodel(order=1, uv=u, xv=x, dxdqv=x, volume=2.0, ndim=1)

In [9]:
xr.testing.assert_allclose(xem.predict(volumes), xem2.predict(volumes))

In [10]:
# resampling
ndat = x.shape[0]
nrep = 20
idx = np.random.choice(ndat, (nrep, ndat), True)

a = xem.resample(indices=idx)#.predict(volumes)
b = xem2.resample(indices=idx)#.predict(volumes)

np.testing.assert_allclose(a.predict(volumes),b.predict(volumes))

In [11]:
# Other format

In [14]:
import thermoextrap.xtrapy.xpan_beta as xpan_beta

In [15]:
data = xpan_beta.DataCentralMomentsVals.from_vals(xv=x, uv=u, order=1, central=False)

/Users/wpk/.conda/envs/py37/lib/python3.7/site-packages/numba/core/dispatcher.py:238: UserWarning: Numba extension module 'sparse._numba_extension' failed to load due to 'ModuleNotFoundError(No module named 'numba.targets')'.
  entrypoints.init_all()


In [17]:
xemd = xpan_vol_ig.factory_extrapmodel_data(volume=2.0, data=data)

In [20]:
xr.testing.assert_allclose(xem.predict(volumes),xemd.predict(volumes))